## Import

In [1]:
import random
import pandas as pd
import numpy as np
import os
from tqdm.auto import tqdm
import librosa

from sklearn.tree import DecisionTreeClassifier

import warnings
warnings.filterwarnings(action='ignore') 

## Hyperparameter Setting

In [2]:
CFG = {
    'SR':16000,
    'N_MFCC':32, # Melspectrogram 벡터를 추출할 개수
    'SEED':42
}

## Fixed Random-Seed

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(CFG['SEED']) # Seed 고정

## Data Pre-Processing

In [4]:
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')

In [5]:
def get_mfcc_feature(df):
    features = []
    for path in tqdm(df['path']):
        # librosa패키지를 사용하여 wav 파일 load
        y, sr = librosa.load(path, sr=CFG['SR'])
        # librosa패키지를 사용하여 mfcc 추출
        mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=CFG['N_MFCC'])
        y_feature = []
        # 추출된 MFCC들의 평균을 Feature로 사용
        for e in mfcc:
            y_feature.append(np.mean(e))
        features.append(y_feature)

    mfcc_df = pd.DataFrame(features, columns=['mfcc_'+str(x) for x in range(1,CFG['N_MFCC']+1)])
    return mfcc_df

In [6]:
train_x = get_mfcc_feature(train_df)
test_x = get_mfcc_feature(test_df)

  0%|          | 0/5001 [00:00<?, ?it/s]

  0%|          | 0/1881 [00:00<?, ?it/s]

In [7]:
train_x['label'] = train_df['label']

## Classification Model Fit

In [12]:
features = list(test_x.columns)
target = 'label'

In [13]:
from catboost import CatBoostClassifier, CatBoostRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from xgboost import XGBClassifier
from sklearn.metrics import log_loss, mean_squared_error, mean_absolute_error, accuracy_score
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import GridSearchCV
from tqdm import tqdm

kf = StratifiedKFold(n_splits = 10)
models = []
val_scores = []
preds = []

params = {
'n_estimators':5000,
'max_depth':6,
'early_stopping_rounds':200,
'learning_rate':0.01,
'one_hot_max_size':3,
'bootstrap_type':"MVS",
'l2_leaf_reg':2,
'random_state':41
         }

for i, (train_index, val_index) in tqdm(enumerate(kf.split(train_x, train_x[target]))):

        # clf.fit(train_x, train_y)

        X_train, X_val = train_x[features].loc[train_index], train_x[features].loc[val_index]
        y_train, y_val = train_x[target][train_index], train_x[target][val_index]

        model = CatBoostClassifier(**params)
    
        model.fit(X_train, y_train, eval_set = [(X_val, y_val)], verbose = 2000)

        models.append(model)
        score = accuracy_score(y_val, model.predict(X_val))
        print(f'''
        ***********************************
            score : {score}                           
        ***********************************
        ''')

        val_scores.append(score)


# model = DecisionTreeClassifier(random_state=CFG['SEED'])
# model.fit(train_x, train_y)

0it [00:00, ?it/s]

0:	learn: 1.7862662	test: 1.7863282	best: 1.7863282 (0)	total: 172ms	remaining: 14m 22s
2000:	learn: 0.8821669	test: 1.2513395	best: 1.2513395 (2000)	total: 29.9s	remaining: 44.9s
4000:	learn: 0.5786088	test: 1.2095490	best: 1.2094785 (3997)	total: 1m	remaining: 15.2s


1it [01:18, 78.38s/it]

4999:	learn: 0.4719526	test: 1.2024885	best: 1.2023982 (4914)	total: 1m 17s	remaining: 0us

bestTest = 1.202398174
bestIteration = 4914

Shrink model to first 4915 iterations.

        ***********************************
            score : 0.5129740518962076                           
        ***********************************
        
0:	learn: 1.7863165	test: 1.7861597	best: 1.7861597 (0)	total: 19.9ms	remaining: 1m 39s
2000:	learn: 0.8772813	test: 1.3160988	best: 1.3160802 (1999)	total: 32.7s	remaining: 49s
4000:	learn: 0.5696727	test: 1.2846023	best: 1.2844654 (3960)	total: 1m 5s	remaining: 16.3s


2it [02:41, 80.94s/it]

...

9it [12:03, 81.01s/it]


        ***********************************
            score : 0.486                           
        ***********************************
        
0:	learn: 1.7862406	test: 1.7865808	best: 1.7865808 (0)	total: 17.1ms	remaining: 1m 25s
2000:	learn: 0.8748923	test: 1.3347750	best: 1.3347750 (2000)	total: 32.8s	remaining: 49.1s
4000:	learn: 0.5653430	test: 1.3086768	best: 1.3084538 (3983)	total: 1m 5s	remaining: 16.4s


10it [13:27, 80.71s/it]

4999:	learn: 0.4602190	test: 1.3050541	best: 1.3049158 (4988)	total: 1m 22s	remaining: 0us

bestTest = 1.304915801
bestIteration = 4988

Shrink model to first 4989 iterations.

        ***********************************
            score : 0.474                           
        ***********************************
        


NameError: name 'train_y' is not defined

In [26]:
preds = np.array([model.predict_proba(test_x[features]) for model in models])
preds

array([[[0.10108775, 0.08525134, 0.02915478, 0.13295887, 0.1783878 ,
         0.47315947],
        [0.61766067, 0.19233677, 0.00300172, 0.07398526, 0.00430061,
         0.10871497],
        [0.00178148, 0.17512822, 0.73262991, 0.03379591, 0.04350131,
         0.01316317],
        ...,
        [0.55884108, 0.04925793, 0.00248856, 0.0753052 , 0.00577423,
         0.308333  ],
        [0.0046205 , 0.31015345, 0.3703508 , 0.09653795, 0.13368893,
         0.08464837],
        [0.03835812, 0.07314498, 0.2738962 , 0.29064208, 0.26016216,
         0.06379646]],

       [[0.08915069, 0.07621068, 0.02266672, 0.14436068, 0.15758557,
         0.51002566],
        [0.61389604, 0.22003886, 0.00143426, 0.06001046, 0.00281468,
         0.1018057 ],
        [0.00272155, 0.23426985, 0.64633789, 0.0476781 , 0.05300195,
         0.01599066],
        ...,
        [0.4663431 , 0.0586023 , 0.00236485, 0.07243724, 0.00301905,
         0.39723346],
        [0.00526381, 0.2889745 , 0.38967191, 0.05929778, 0.177

In [27]:
preds.mean(axis=0)

array([[0.09872608, 0.0800743 , 0.02740238, 0.14478668, 0.18508593,
        0.46392463],
       [0.61934247, 0.20428827, 0.00242401, 0.06321666, 0.00515744,
        0.10557115],
       [0.00211691, 0.22260054, 0.67332162, 0.04005711, 0.04642907,
        0.01547475],
       ...,
       [0.59273301, 0.04997801, 0.00283236, 0.07502555, 0.00475413,
        0.27467695],
       [0.00540109, 0.26819583, 0.36719881, 0.09929249, 0.18884381,
        0.07106796],
       [0.03481494, 0.08451619, 0.30743354, 0.24134658, 0.27453459,
        0.05735416]])

In [28]:
preds = [np.argmax(i) for i in preds.mean(axis=0)]

## Inference

In [31]:
pd.DataFrame(preds).value_counts()

2    398
0    372
5    315
3    296
4    288
1    212
dtype: int64

## Submission

In [29]:
submission = pd.read_csv('./sample_submission.csv')
submission['label'] = preds
submission.to_csv('./baseline_submission.csv', index=False)